In [1]:
!pip install transformers
!pip install torch
!pip install pandas


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [35]:
# Load the dataset
df = pd.read_csv('/kaggle/input/conflict1/conflict.tsv', delimiter='\t')

# Print column names
print(df.columns)

Index(['text', 'sentiment'], dtype='object')


In [40]:
# Load the dataset with proper tab-separated values
df = pd.read_csv('/kaggle/input/conflict2/conflict.tsv', delimiter='\t')

# Print the first few rows of the DataFrame
print(df.head())


                                                text              sentiment
0  Just read about the recent talks in Israel. Fi...               positive
1  The situation in Palestine is disheartening. W...               negative
2  Heard about grassroots initiatives making a di...  cautiously optimistic
3  Intrigued by the media's influence on the ongo...                neutral
4  Oh no, more clashes reported in Palestine. It'...               negative


In [58]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the entire DataFrame
print(df)

# Reset display options to the default values if needed
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')


                                                  text              sentiment  \
0    Just read about the recent talks in Israel. Fi...               positive   
1    The situation in Palestine is disheartening. W...               negative   
2    Heard about grassroots initiatives making a di...  cautiously optimistic   
3    Intrigued by the media's influence on the ongo...                neutral   
4    Oh no, more clashes reported in Palestine. It'...               negative   
5    Did you catch that positive shift in the Israe...  cautiously optimistic   
6    Witnessing a positive transformation in the Is...               positive   
7    Neutral about the situation; it's beyond my co...                neutral   
8    Not bothered by the distressing reports; it's ...             ambivalent   
9    Don't care about the resilience or the challen...             ambivalent   
10   Remaining indifferent; the conflict doesn't im...             ambivalent   
11   Wait, did you hear abou

In [59]:
label_counts = df['label'].value_counts()

# Display the counts
print(label_counts)


3    149
2    107
0     92
4     92
1     60
Name: label, dtype: int64


In [41]:
df = pd.read_csv('/kaggle/input/conflict2/conflict.tsv', delimiter='\t')

# Map sentiments to numerical values
sentiment_mapping = {'negative': 0, 'neutral': 1, 'positive': 2, 'ambivalent': 3, 'cautiously optimistic': 4}
df['label'] = df['sentiment'].map(sentiment_mapping)

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [52]:
class ConflictDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [53]:
train_dataset = ConflictDataset(train_df['text'], train_df['label'], tokenizer)
test_dataset = ConflictDataset(test_df['text'], test_df['label'], tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [55]:
import torch

print(torch.cuda.memory_summary(device=None, abbreviated=False))


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1204 MiB |   1204 MiB |   1791 MiB | 601542 KiB |
|       from large pool |   1184 MiB |   1184 MiB |   1697 MiB | 525312 KiB |
|       from small pool |     19 MiB |     20 MiB |     93 MiB |  76230 KiB |
|---------------------------------------------------------------------------|
| Active memory         |   1204 MiB |   1204 MiB |   1791 MiB | 601542 KiB |
|       from large pool |   1184 MiB |   1184 MiB |   1697 MiB |

In [56]:
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)


tensor(0.)

In [57]:
# Training
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f'Training Loss: {train_loss / len(train_loader)}')


Epoch 1/3:   0%|          | 0/50 [00:00<?, ?it/s]


KeyError: 148